# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [280]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # cleaning the cities dataframe
cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                           "KingsDucks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings", 
                                           "Maple Leafs": "Maple,Leafs", 
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights" })
cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
cities = cities.explode("NHL")
cities["NHL"] = cities["NHL"] = cities["NHL"].apply(lambda x: x.split(" ")[-1])

    # cleaning the nhl_df dataframe
nhl_df = nhl_df[nhl_df["year"] == 2018]
nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", ""))
nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])

    # merge the dataframes
df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
df["W-L%"] = df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
df["W-L%"] = df["W-L%"].astype("float")

# drop duplicated columns
df.loc[df["Metropolitan area"] == "New York City", "W-L%"] = 0.5182013333333334
df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.6228945
df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

def nhl_correlation(): 
    population_by_region = df["Population (2016 est.)[8]"]
    win_loss_by_region = df["W-L%"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]
nhl_correlation()

0.012485959345532895

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [249]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
nba_df = nba_df.replace(to_replace="[*].*", value="", regex=True)
nba_df
cities = cities.replace(to_replace="[[]note.*", value="", regex=True)
cities["NBA"] = cities["NBA"].apply(lambda x: x.split(" "))
cities = cities.explode("NBA")
nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])
nba_df["team"] = nba_df["team"].apply(lambda x: x.split("(")[0])
nba_df["team"].replace(to_replace = "\s*", value="", regex=True,inplace=True)
nba_df = nba_df[nba_df["year"] == 2018]
df_2 = pd.merge(nba_df, cities, left_on = "team", right_on = "NBA")
df_2["Population (2016 est.)[8]"] = df_2["Population (2016 est.)[8]"].astype("float")
df_2["W/L%"] = df_2["W/L%"].astype("float")
df_2=df_2.reset_index()
df_2.loc[df_2["Metropolitan area"] == "New York City", "W/L%"] = 0.34750000000000003
df_2.loc[df_2["Metropolitan area"] == "Los Angeles", "W/L%"] = 0.46950000000000003
df_2 = df_2.drop_duplicates(subset="Metropolitan area").reset_index()
df_2 = df_2.drop(columns="index")
    
def nba_correlation():
    
    population_by_region_2 = df_2["Population (2016 est.)[8]"]
    win_loss_by_region_2 = df_2["W/L%"]

    assert len(population_by_region_2) == len(win_loss_by_region_2), "Q2: Your lists must be the same length"
    assert len(population_by_region_2) == 28, "Q2: There should be 28 teams being analysed for NBA"

    result = stats.pearsonr(population_by_region_2, win_loss_by_region_2)
    return result[0]
#nba_correlation()

-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [356]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#mlb_df #0 RED SOX; #8 WHITE SOX
cities["MLB"] = cities["MLB"].replace(to_replace = "[[]note \d*[]]", value="", regex=True)
cities["MLB"] = cities["MLB"].apply(lambda x: x.split(" ")[-1])
cities["MLB"][(cities["Metropolitan area"] == "Chicago")&(cities["MLB"] == "Sox")] = "White Sox"
cities["MLB"][(cities["Metropolitan area"] == "Boston")&(cities["MLB"] == "Sox")] = "Red Sox"
mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])
mlb_df = mlb_df[mlb_df["year"] == 2018]
mlb_df["team"][(mlb_df["team"] == "Sox")&(mlb_df["W"] == 108)] = "Red Sox"
mlb_df["team"][(mlb_df["team"] == "Sox")&(mlb_df["W"] == 62)] = "White Sox"
df_3 = pd.merge(mlb_df, cities, left_on = "team", right_on = "MLB")
df_3["Population (2016 est.)[8]"] = df_3["Population (2016 est.)[8]"].astype("float")
df_3["W-L%"] = df_3["W-L%"].astype("float")
df_3.loc[df_3["Metropolitan area"] == "New York City", "W-L%"] = 0.546
df_3.loc[df_3["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.5289999999999999
df_3.loc[df_3["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.525
df_3.loc[df_3["Metropolitan area"] == "Chicago", "W-L%"] = 0.482769
df_3.loc[df_3["Metropolitan area"] == "Boston", "W-L%"] = 0.666667


def mlb_correlation(): 
    # YOUR CODE HERE
    
    population_by_region = df_3["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df_3["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.14999221533065174

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [388]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nfl_df = nfl_df[nfl_df["year"] == 2018]
nfl_df
cities=cities[["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
cities = cities.replace(to_replace = "[[].*[]]", value="",regex=True)
#cities
nfl_df = nfl_df.replace(to_replace = "[*]", value= "", regex=True)
nfl_df = nfl_df.replace(to_replace = "[+]", value= "", regex=True)
cities["NFL"][cities["NFL"] == "49ers Raiders"] = "49ers"
nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])
cities["NFL"] = cities["NFL"].apply(lambda x: x.split(" ")[-1])
df_4 = pd.merge(nfl_df, cities, left_on = "team", right_on = "NFL")
df_4["Population (2016 est.)[8]"] = df_4["Population (2016 est.)[8]"].astype("float")
df_4["W-L%"] = df_4["W-L%"].astype("float")
df_4.loc[df_4["Metropolitan area"] == "New York City", "W-L%"] = 0.2815 # mean of NY W-L%
df_4.loc[df_4["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.7815 # mean of LA W-L%
df_4.loc[df_4["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.25 # mean of SF W-L%

def nfl_correlation(): 
    # YOUR CODE HERE
    
    population_by_region = df_4["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df_4["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

-0.03439109872465536

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [376]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
#nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
nfl_df

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018
5,AFC North,AFC North,NFL,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,2018
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens,2018
7,1.7,6,NFL,4.3,3.9,360,68,428,5.6,1.3,1,9,.594,Pittsburgh Steelers,2018
8,0.6,8,NFL,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7,.469,Cleveland Browns,2018
9,-3.4,10,NFL,-5.4,0.0,455,-87,368,-3.4,2.0,0,6,.375,Cincinnati Bengals,2018
